# Parcel Logistics

## Table of Contents

1. [Introduction](#introduction)
1. [Mathematical Model](#model)
1. [Data Source and Format](#data)
    1. [Data Source](#data_source)
    1. [Data Format](#data_format)
1. [Implementation](#implementation)
1. [Conclusion](#conclustion)

<a name="introduction"></a>
### 1. Introduction ###

<a name="model"></a>
### 2. Mathematical Model ###

<a name="data"></a>
### 3. Data Source and Format ###

<a name="data_source"></a>
#### 3.1 Data Source ####

The data source we are using in this project is from a GitHub repository, https://github.com/austinlasseter/datasets-shipping-logistics


<a name="data_format"></a>
#### 3.2 Data Format ####

<a name="implementation"></a>
### 4. Implementation ###

In [ ]:
# Install Ipopt
using Pkg
Pkg.add("Ipopt")
Pkg.add("PyPlot")

using JuMP, Ipopt
using PyPlot

<a name="conclusion"></a>
### 5. Conclusion ###